# 0、准备工作

In [1]:
import requests,urllib,urllib.error,json
from bs4 import BeautifulSoup #页面解析，获取数据
import re
import xlwt

##  得到一个指定内容的url网页

https://www.lottery.gov.cn/kj/kjlb.html?dlt  原网页

In [2]:
def askURL(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"
    }
    request = urllib.request.Request(url,headers=headers)
    html = ""
    try:
        reponse = urllib.request.urlopen(request)
        html = reponse.read().decode('utf-8')
        #print(html)
    except urllib.error.URLError as e:
        if hasattr(e,"code"):
            print(e.code)
        if hasattr(e,"reason"):
            print(e.reason)
    return html

# 1、爬取网页   2、解析数据   

In [3]:
baseurl = "https://webapi.sporttery.cn/gateway/lottery/getHistoryPageListV1.qry?gameNo=85&provinceId=0&pageSize=30&isVerify=1&pageNo="

In [4]:
def getData(baseurl):
    datalist = []
    for item in range(1,71):
        url = baseurl +str(item)
        
        html = askURL(url)   #保存获取到的网页源码
        
        #逐一解析数据
        js = json.loads(html)
        for i in js['value']['list']:
            data = []
            lotteryDrawNum = i['lotteryDrawNum']
            data.append(lotteryDrawNum)
            lotteryDrawTime = i['lotteryDrawTime']
            data.append(lotteryDrawTime)
            lotteryDrawResult = i['lotteryDrawResult']
            data.append(lotteryDrawResult)
            #print(data)
            datalist.append(data)
    print("爬取完毕！")
        
    return datalist


# 3、保存数据

In [5]:
def saveData(datalist,savepath):
    book = xlwt.Workbook(encoding='utf-8',style_compression=0)  #创建book对象
    sheet = book.add_sheet('大乐透',cell_overwrite_ok = True)   #创建工作表
    col = ('期数',"日期","前区1","前区2","前区3","前区4","前区5","后区1","后区2")
    for i in range(0,len(col)):
        sheet.write(0,i,col[i])  #列名
    for i in range(0,len(datalist)):
        data = datalist[i]
        sheet.write(i+1,0,data[0])    #期数
        sheet.write(i+1,1,data[1])    #日期
        for j in range(2,len(col)):   #数字
            sheet.write(i+1,j,data[2].split()[j-2])
    
    book.save('大乐透.xlsx')

# 4、执行

In [6]:
datalist = getData(baseurl)

爬取完毕！


In [7]:
savepath = './'
saveData(datalist,savepath)